In [ ]:
import urllib.request as urllib
from bs4 import BeautifulSoup
import re
from contextlib import closing
import sqlite3
import time
from selenium.webdriver.chrome.options import Options
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait



階層構造にしないので、レビューDBは巨大なものにする。  
全部一つにまとめる  
レヴュアーカラムはレビューの数だけ同じものを入れる  

このあとやること
HTML検索はbs4でやる
スクロールはseleniumでやる

参考URL
https://shotanuki.com/%E5%8B%95%E7%9A%84%E3%83%9A%E3%83%BC%E3%82%B8%E3%81%AE%E3%82%B9%E3%82%AF%E3%83%AC%E3%82%A4%E3%83%94%E3%83%B3%E3%82%B0%E3%82%92selenium%E3%81%A7%E3%82%84%E3%81%A3%E3%81%A6%E3%81%BF%E3%81%9F/

In [ ]:
url='https://www.amazon.co.jp/gp/profile/amzn1.account.AFBR6VVHA2M3PXU5V24TNBMUWYOA/ref=cm_cr_getr_d_gw_btm?ie=UTF8'
options = Options()
options.add_argument('--headless')
 
browser = webdriver.Chrome(chrome_options=options) 


In [ ]:
def scroll_and_get_100_Products_url(url):
    '''
    レビュアー詳細URLからその人の全レビューURLを取得するコード
    その人の獲得した星の数や、レビュー数もその気になれば手に入れられる
    100件以上は収集しない
    '''
    browser.get(url)
    time.sleep(0.3) 
    soup = BeautifulSoup(browser.page_source, 'html.parser')
    review_number=soup.select('.dashboard-desktop-stat-value')
    print(review_number)
    review_box=soup.select('.profile-at-card')
    review_url=['https://www.amazon.co.jp'+x.select('.profile-at-review-link')[0].get('href') for x in review_box]
    i=0
    try:
        print(review_number[1].text)
    except:
        browser.execute_script('scroll(0, document.body.scrollHeight)')
        time.sleep(0.3)
        soup = BeautifulSoup(browser.page_source, 'html.parser')
        review_number=soup.select('.dashboard-desktop-stat-value')
        print(review_number)
        review_box=soup.select('.profile-at-card')
        review_url=['https://www.amazon.co.jp'+x.select('.profile-at-review-link')[0].get('href') for x in review_box]
    
    while(int(review_number[1].text)!=len(review_url)):
        i+=1
        if(len(review_url)>=100):
            break
        if i==10:
            break
        browser.execute_script('scroll(0, document.body.scrollHeight)')
        time.sleep(0.3) 
        soup = BeautifulSoup(browser.page_source, 'html.parser')
        review_number=soup.select('.a-size-large.a-color-base')
        review_box=soup.select('.profile-at-card')
        review_url=['https://www.amazon.co.jp'+x.select('.profile-at-review-link')[0].get('href') for x in review_box]
        print(f'{len(review_url)}件のレビューを取得しました。')
    
    return(review_url)

#商品レビューURLから商品URLに行き、カテゴリーをリスト型にして持ってきてくれる
def get_category(product_url):
    browser.get(product_url)
    soup = BeautifulSoup(browser.page_source, 'html.parser')
    category=soup.select('.a-unordered-list')[0].text
    category=re.sub('›','',category)
    category=re.sub('\n','',category)
    
    category.split()
    return(category)




In [ ]:
def get_each_product_info(url):
    '''
    レビュー詳細URLから必要な情報を手に入れるコード
    SQLに紐づけて必要な部分を後から埋めるでもいいかもしれない。
    categoryの数が変動するので注意
    '''
    url2='https://www.amazon.co.jp/gp/customer-reviews/R1JYHF8TMEKK3J?ref=pf_vv_at_pdctrvw_srp'
    browser.get(url2)
    soup = BeautifulSoup(browser.page_source, 'html.parser')
    title=soup.select('.review-title')[0].text
    star=float(re.sub('5つ星のうち','',soup.select('.a-icon-alt')[0].text))
    review=soup.select('.review-text')[0].text
    product_url='https://www.amazon.co.jp'+soup.select('.a-text-ellipsis > a')[0].get('href')
    category=get_category(product_url)
    return(title,star,review,category)

def get_db_conn(name):
    '''
    sql文を何度も書きたくなかったので書いた
    '''
    conn=sqlite3.connect(name)
    c=conn.cursor()
    return(conn,c)

def get_one_people_each_products_url(Rconn,Rc,Wconn,Wc,select_sql):
    '''
    レビュワーURLを一気に受け取り、一つずつfor文を回す。
    その後一度にレビューURLを取得し、新しいDBに格納する
    '''
    for i,row in enumerate(Rc.execute(select_sql)):
        print(f'{i}人目、{row[0]}さんのレビューを取得中です')

        product_url=scroll_and_get_100_Products_url('https://www.amazon.co.jp'+row[2])
        name_list=[row[0]]*len(product_url)
        rate_list=[row[1]]*len(product_url)
        id_=[x for x in range(1,len(product_url)+1)]
        datas=[(n,r,i,p) for n,r,i,p in zip(name_list,rate_list,id_,product_url)]

        insert_sql = 'insert into products (name,id,rate,url) values (?,?,?,?)'
        #↑順番が違う
        Wc.executemany(insert_sql, datas)
        Wconn.commit()



In [ ]:
#ある人のレビュワーページから全部のレビュー詳細URLを手に入れるコード。
url='https://www.amazon.co.jp/gp/profile/amzn1.account.AHXU2TX6WIQFKLQWKOZBP322S76Q/ref=cm_cr_dp_d_gw_tr?ie=UTF8'
road_dbname='1105_review_of_bakawall7.db'
low_dbname='1107_low_rate_bakawall_each_prodcuts.db'
high_dbname='1107_high_rate_bakawall_each_prodcuts.db'

    

def main():
    Rconn,Rc=get_db_conn(road_dbname)
    low_select_Rdb=f'''select date,rate,url from reviews where rate<3'''
    high_select_Rdb=f'''select date,rate,url from reviews where rate>3'''
    
    
    Hconn,Hc=get_db_conn(high_dbname)
    Lconn,Lc=get_db_conn(low_dbname)
    head='https://www.amazon.co.jp'
    
#     create_table = '''create table products (id,name,rate,url,review,text)'''
#     Hc.execute(create_table)
#     Lc.execute(create_table)
    
    get_one_people_each_products_url(Rconn,Rc,Hconn,Hc,high_select_Rdb)
    get_one_people_each_products_url(Rconn,Rc,Lconn,Lc,low_select_Rdb)




# soup = BeautifulSoup(browser.page_source, 'html.parser')
# review_number=soup.select('.a-size-large.a-color-base')
# review_box=soup.select('.profile-at-card')
# review_url=['https://www.amazon.co.jp'+x.select('.profile-at-review-link')[0].get('href') for x in review_box]
# print(int(review_number[1].text))
# print(review_url)

In [ ]:
main()

これより下実験コード↓

In [ ]:
Rconn,Rc=get_db_conn(road_dbname)
Rc.execute('''select * from sqlite_master''')
for r in Rc:
    print(r)
Rc.execute('''select * from reviews''')
for r in Rc:
    print(r)
    

In [ ]:
url='https://www.amazon.co.jp/gp/profile/amzn1.account.AFBR6VVHA2M3PXU5V24TNBMUWYOA/ref=cm_cr_getr_d_gw_btm?ie=UTF8'
headers = {
        "User-Agent": "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:47.0) Gecko/20100101 Firefox/47.0",
        }
request = urllib.Request(url,headers=headers)
html = urllib.urlopen(request)

soup = BeautifulSoup(html,'lxml')
print(soup)
# review_number=soup.select('.a-size-large.a-color-base')
# print(review_number)
        

In [ ]:
#レビューカラム数が最初のレビュー数に一致したらbreakする

url='https://www.amazon.co.jp/gp/profile/amzn1.account.AFBR6VVHA2M3PXU5V24TNBMUWYOA/ref=cm_cr_getr_d_gw_btm?ie=UTF8'
browser.execute_script('window.scrollBy(0,160)', '')
options = Options()
options.add_argument('--headless')
 
browser = webdriver.Chrome(chrome_options=options) 

def scroll_down(url):
    browser.get(url)
    scroll_count=0
    for i in range(0, 200):
        if scroll_count == 0:
            time.sleep(1)
        link_list = []
        soup = BeautifulSoup(browser.page_source, 'html.parser')
        soup.find_all()


    


browser.close()

In [ ]:
#DB使ってfor文でまとめて行う。、その前に一つだけ使って考える↑
